In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from gurobipy import *

In [ ]:
# load profile
pd.options.plotting.backend = "plotly"
template = "plotly_white"
#profile = pd.read_csv("../data/ecostor_fcr_wholesale_15min_3.csv", index_col=0, parse_dates=True)
profile = pd.read_csv("../data/SBAP_FCR_profile_test2.csv", index_col=0, parse_dates=True)
#path="../data/ecostor_fcr_wholesale_15min_3.csv"
#profile = pd.read_csv("../data/industry_profile_test.csv", index_col=0, parse_dates=True)
#profile = pd.read_csv("../data/myProfile3.csv", index_col=0, parse_dates=True)
profile.plot(template=template, labels={"value": "Power [W]"})

In [ ]:
# Storage parameters
storage_params_1 = {
    "capacity": 1500,  # kWh
    #"power": 150.0,      # kW
    "power": 600,
    
    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.6,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}
storage_params_2 = {
    "capacity": 1500,  # kWh
    #"power": 150.0,      # kW
    "power": 600,

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.3,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

# Initialization of Power profile
initial_inputs = {
    "timestep": 0.25,   #15min timesteps
    "p_ch1": 0,
    "p_dch1": 0,
    "p_ch2": 0,
    "p_dch2": 0,
    "kw": 0.001   #W to kW conversion, Make it equal to "1.0" if the input profile is in kW
    
}

In [ ]:
 ## =====  Parameters  ===== ##
n = len(profile)   # number of timesteps
time = range(0, n-1) # T
dt = initial_inputs["timestep"]               # 15 min timesteps
kw= initial_inputs["kw"]

## Battery1
capacity1  = storage_params_1["capacity"]
max_power1 = storage_params_1["power"]
p_ch1=initial_inputs["p_ch1"]
p_dch1=initial_inputs["p_dch1"]

soc1_min, soc1_max = storage_params_1["soc_bounds"]
soc1_range= soc1_max-soc1_min 


effc1  = storage_params_1["effc"]
effd1  = storage_params_1["effd"]

 ## Battery2

capacity2  = storage_params_2["capacity"]
max_power2 = storage_params_2["power"]

p_ch2=initial_inputs["p_ch2"]
p_dch2=initial_inputs["p_dch2"]

soc2_min= 0.1
soc2_max = 0.9
soc2_range= soc2_max-soc2_min 


effc2  = storage_params_2["effc"]
effd2 = storage_params_2["effd"]

pe_1=1
pe_2=1


   ## =====  Variables  ===== ##
# BESS1: power charge/discharge, energy content
soc1             = [0.0] * len(time)
power_tu1        = [0.0] * len(time)
power_charge1    = [0.0] * len(time)
power_discharge1 = [0.0] * len(time)
energy_bess1     = [0.0] * len(time)
delta_SOC        = [0.0] * len(time)
lump_delta_SOC   = [0.0] * len(time)
power_loss1      = [0.0] * len(time)

 # BESS2: power charge/discharge, energy content
soc2             = [0.0] * len(time)
power_tu2        = [0.0] * len(time)
power_charge2    = [0.0] * len(time)
power_discharge2 = [0.0] * len(time)
energy_bess2     = [0.0] * len(time)
abs_delta        = [0.0] * len(time)   
power_loss2      = [0.0] * len(time)

AI = [1.0] * len(time)               #Availability index

soc1[0] = storage_params_1["soc_start"]
soc2[0] = storage_params_2["soc_start"]

zero = [0.0] * len(time)
frac = [0.01] * len(time)
kw= initial_inputs["kw"]  

In [ ]:
for t in range(1,len(time)):
  #timestep = time[t]-time[t-1]
  #print('timestep',t)
    delta_SOC[0]= abs(soc2[0]-soc1[0])
    lump_delta_SOC[0]=abs(soc2[0]-soc1[0])
  # soc1[1]=soc1_ini
  # soc2[1]=soc2_ini
  #Need to still implement the set point overriding scenario if power_tu1 > max_power1
  #if soc1_min<soc1[t-1]<soc1_max and soc2_min<soc2[t-1]<soc2_max:
    if abs(profile.power[t-1])>max_power1:
      print('Power demand >= single string nominal power')
      #print('SoC Format: Delta_SoC, soc1, soc2')
      print('start',(soc2[t-1]-soc1[t-1]),soc1[t-1],soc2[t-1])
      if profile.power[t-1]>=0: #charging case
          power_tu1[t-1]=(0.5+((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
          power_tu2[t-1]=(0.5-((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
          power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
          power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
          soc1[t]=soc1[t-1]+(((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
          soc2[t]=soc2[t-1]+(((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
          delta_SOC[t]= abs(soc2[t]-soc1[t])
          lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
          pe_1=pe_1+1
          pe_2=pe_2+1

          if soc1[t]>soc1_max or soc2[t]>soc2_max:
            soc1[t]=soc1[t-1]
            soc2[t]=soc2[t-1]
            power_tu1[t-1]=0
            power_tu2[t-1]=0
            power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
            power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
            AI[t-1]= 0

          print('Str1 and Str2 Charging: Power_Strings:', power_tu1[t-1], power_tu2[t-1])
          print('Power_Loss:', power_loss1[t-1])
            
      else: #power_tu1[t-1]<0: discharging case
          #print('Discharging',(soc2[t-1]-soc1[t-1]),soc1[t-1],soc2[t-1])
          power_tu1[t-1]=(0.5-((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
          power_tu2[t-1]=(0.5+((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
          power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
          power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
          soc1[t]=soc1[t-1]+(((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
          soc2[t]=soc2[t-1]+(((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
          delta_SOC[t]= abs(soc2[t]-soc1[t])
          lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
          pe_1=pe_1+1
          pe_2=pe_2+1

          if soc1[t]<soc1_min or soc2[t]<soc2_min:
            soc1[t]=soc1[t-1]
            soc2[t]=soc2[t-1]
            power_tu1[t-1]=0
            power_tu2[t-1]=0
            power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
            power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
            AI[t-1]= 0
    else:
       print('Power demand < single string nominal power')
       if profile.power[t-1]>=0: #charging case
          if ((soc1[t-1]<=soc1_min and soc2_min<soc2[t-1]<soc2_max) or (soc1_min<soc1[t-1]<soc1_max and soc2[t-1]>=soc2_max) or (soc1[t-1]<=soc1_min and soc2[t-1]>=soc2_max)): #mode2,5,8
            power_tu1[t-1]=profile.power[t-1]*kw
            power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
            soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))

            power_tu2[t-1]=0
            power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
            soc2[t]=soc2[t-1]
            
            delta_SOC[t]= abs(soc2[t]-soc1[t])
            lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
            pe_1=pe_1+1
          elif ((soc1_min<soc1[t-1]<soc1_max and soc2[t-1]<=soc2_min)or(soc1[t-1]>=soc1_max and soc2_min<soc2[t-1]<soc2_max)or(soc1[t-1]>=soc1_max and soc2[t-1]<=soc2_min)): #mode3,4,9
            power_tu2[t-1]=profile.power[t-1]*kw
            power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
            soc2[t]=soc2[t-1]+((power_tu2[t-1]*dt)/(capacity2)) - ((power_loss1[t-1]*dt)/(capacity1))
            
            power_tu1[t-1]=0
            power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
            soc1[t]=soc1[t-1]
            delta_SOC[t]= abs(soc2[t]-soc1[t])
            lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
            pe_2=pe_2+1
          elif ((soc1[t-1]<=soc1_min and soc2[t-1]<=soc1_min)or(soc1[t-1]>=soc1_max and soc2[t-1]>=soc1_max)): #mode 6and7
            if (soc2[t-1]-soc1[t-1])<= -0.1:
              power_tu2[t-1]=profile.power[t-1]*kw
              power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
              soc2[t]=soc2[t-1]+((power_tu2[t-1]*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity1))
              
              power_tu1[t-1]=0
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
              soc1[t]=soc1[t-1]

              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_2=pe_2+1

              if soc2[t]>soc2_max:
                power_tu2[t-1]=0
                power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
                soc2[t]=soc2[t-1]
                AI[t-1]= 0

              print('Str2 Charging: Power_String:', power_tu2[t-1])
        
            elif (soc2[t-1]-soc1[t-1])>= 0.1:
              power_tu1[t-1]=profile.power[t-1]*kw
              soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1))- ((power_loss1[t-1]*dt)/(capacity1))
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))

              soc2[t]=soc2[t-1]
              power_tu2[t-1]=0
              power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))


              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_1=pe_1+1

              if soc1[t]>soc1_max:
                soc1[t]=soc1[t-1]
                power_tu1[t-1]=0
                power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
                AI[t-1]= 0
    
            else:
              power_tu1[t-1]=(0.5+((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
              power_tu2[t-1]=(0.5-((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1)) 
              power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power1))
              soc1[t]=soc1[t-1]+(((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
              soc2[t]=soc2[t-1]+(((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_1=pe_1+1
              pe_2=pe_2+1

              if soc1[t]>soc1_max or soc2[t]>soc2_max:
                soc1[t]=soc1[t-1]
                soc2[t]=soc2[t-1]
                power_tu1[t-1]=0
                power_tu2[t-1]=0
                power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1)) 
                power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power1))
                AI[t-1]= 0

          elif(soc1_min<soc1[t-1]<soc1_max and soc2_min<soc2[t-1]<soc2_max):   #mode:1
            if ((t-1)%24)==0:
              power_tu1[t-1]=profile.power[t-1]*kw
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
              soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
              
              power_tu2[t-1]=0
              soc2[t]=soc2[t-1]
              power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_1=pe_1+1
              if soc1[t]>soc1_max:
                soc1[t]=soc1[t-1]
                power_tu1[t-1]=0
                power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
                AI[t-1]= 0
            else:
              power_tu2[t-1]=profile.power[t-1]*kw
              power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power2))
              soc2[t]=soc2[t-1]+((power_tu2[t-1]*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
              
              power_tu1[t-1]=0
              soc1[t]=soc1[t-1]
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_2=pe_2+1
              if soc2[t]>soc2_max:
                power_tu2[t-1]=0
                soc2[t]=soc2[t-1]
                power_loss2[t-1]= (0.021*(power_tu2[t-1])+(0.005*max_power1))
                AI[t-1]= 0
              print('Str2 Charging: Power_String:', power_tu2[t-1])
              print('Power_Loss:', power_loss2[t-1])
              #p_loss[t-1]=(((2*power_tu2[t-1]/max_power1)-0.8)*0.01)*(power_tu2[t-1])
              #print('Switch Strings: Mode 1',(soc2[t]-soc1[t]), soc1[t], soc2[t])
      
       else: #discharging case
            print('Discharging',(soc2[t-1]-soc1[t-1]),soc1[t-1],soc2[t-1])
            if ((soc1[t-1]<=soc1_min and soc1_min<soc2[t-1]<soc1_max) or (soc1_min<soc1[t-1]<soc1_max and soc2[t-1]>=soc1_max) or (soc1[t-1]<=soc1_min and soc2[t-1]>=soc1_max)): #mode2,5,8
              power_tu2[t-1]=profile.power[t-1]*kw
              power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power1))
              soc2[t]=soc2[t-1]+((power_tu2[t-1]*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
              power_tu2[t-1]=0
              soc1[t]=soc1[t-1]
              power_loss1[t-1]= (0.021*(power_tu1[t-1])+(0.005*max_power1))
              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_2=pe_2+1

            elif ((soc1_min<soc1[t-1]<soc1_max and soc2[t-1]<=soc1_min)or(soc1[t-1]>=soc1_max and soc1_min<soc2[t-1]<soc1_max)or(soc1[t-1]>=soc1_max and soc2[t-1]<=soc1_min)): #mode3,4,9
              power_tu1[t-1]=profile.power[t-1]*kw
              power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
              soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
              
              soc2[t]=soc2[t-1]
              power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power1))
              delta_SOC[t]= abs(soc2[t]-soc1[t])
              lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
              pe_1=pe_1+1
              print('Str1 Charging: Power_String:', power_tu1[t-1])
              print('Power_Loss:', power_loss1[t-1])

            elif ((soc1[t-1]<=soc1_min and soc2[t-1]<=soc2_min)or(soc1[t-1]>=soc1_max and soc2[t-1]>=soc2_max)): #mode 6and7
              if (soc2[t-1]-soc1[t-1])<= -0.1:
                power_tu1[t-1]=profile.power[t-1]*kw
                power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
                
                soc2[t]=soc2[t-1]
                power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power1))
                delta_SOC[t]= abs(soc2[t]-soc1[t])
                lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
                pe_1=pe_1+1

                if soc1[t]<soc1_min:
                  soc1[t]=soc1[t-1]
                  power_tu1[t-1]=0
                  power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                  AI[t-1]= 0

              elif (soc2[t-1]-soc1[t-1])>= 0.1:
                power_tu2[t-1]=profile.power[t-1]*kw
                power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
                soc2[t]=soc2[t-1]+((power_tu1[t-1]*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
                
                
                power_tu1[t-1]=0
                power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                soc1[t]=soc1[t-1]
                
                delta_SOC[t]= abs(soc2[t]-soc1[t])
                lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
                pe_2=pe_2+1

                if soc2[t]<soc2_min:
                  soc2[t]=soc2[t-1]
                  power_tu2[t-1]=0
                  power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
                  AI[t-1]= 0

              else:
                power_tu1[t-1]=(0.5-((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
                power_tu2[t-1]=(0.5+((soc2[t-1]-soc1[t-1])*0.5))*profile.power[t-1]*kw
                
                power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1)) 
                power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))

                soc1[t]= soc1[t-1]+ (((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
                soc2[t]= soc2[t-1]+ (((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))

                delta_SOC[t]= abs(soc2[t]-soc1[t])
                lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
                pe_1=pe_1+1
                pe_2=pe_2+1

                if soc1[t]<soc1_min or soc2[t]<soc2_min:
                  soc1[t]=soc1[t-1]
                  soc2[t]=soc2[t-1]
                  power_tu1[t-1]=0
                  power_tu2[t-1]=0
                  power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                  power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
                  AI[t-1]= 0

            elif(soc1_min<soc1[t-1]<soc1_max and soc1_min<soc2[t-1]<soc1_max): #Mode1
              if ((t-1)%24)==0:
                power_tu1[t-1]=profile.power[t-1]*kw
                power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
                soc1[t]=soc1[t-1]+((power_tu1[t-1]*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
                soc2[t]=soc2[t-1]
                delta_SOC[t]= abs(soc2[t]-soc1[t])
                lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
                pe_1=pe_1+1

              else:
                power_tu2[t-1]=profile.power[t-1]*kw
                power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
                soc2[t]=soc2[t-1]+((power_tu2[t-1]*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
                soc1[t]=soc1[t-1]
                power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
                delta_SOC[t]= abs(soc2[t]-soc1[t])
                lump_delta_SOC[t] = lump_delta_SOC[t-1] + delta_SOC[t]
                pe_2=pe_2+1
    
  #else:
  #  print ("Time: ", t)
  #  #print("2.Batteries breached limit")
  #  soc1[t]=soc1[t-1]
  #  soc2[t]=soc2[t-1]
  #  AI[t-1]= 0
  #  power_tu1[t-1]=0
  #  power_tu2[t-1]=0
  #  power_loss1[t-1] = (0.021*(power_tu1[t-1])+(0.0075*max_power2))
  #  power_loss2[t-1] = (0.021*(power_tu2[t-1])+(0.0075*max_power2))          

In [ ]:

# Data
SoE1 = [soc1[t] for t in time]
SoE2 = [soc2[t] for t in time]

del_SOE=[abs(soc2[t]-soc1[t]) for t in time]

# Create a new figure
fig, ax = plt.subplots(figsize=(15, 8))

# Plot the data
ax.plot(profile.index[:len(SoE1)], SoE1, label="String_1_SOC", color="orange", linewidth=2)
ax.plot(profile.index[:len(SoE2)], SoE2, label="String_2_SOC", color="green", linewidth=2,linestyle='--')

#ax2 = ax.twinx()
#ax2.plot(profile.index[:len(del_SOE)], del_SOE, label="del_SOC", color="brown", linewidth=1)

# Set labels and title
ax.set_xlabel("Time (days)",fontsize=34)
ax.set_ylabel("State of Charge",fontsize=34)
#ax.set_title("State of Charge Over Time")
ax.legend(fontsize=24, loc='lower right')
# Add grid lines
ax.grid(True, linestyle='--')

# Set labels and title for the second y-axis
#ax2.set_ylabel("delta_SOC", fontsize=24)
ax.tick_params(axis='y', labelsize=26)
ax.tick_params(axis='x', labelsize=26)
plt.yticks(np.arange(0,1.0, 0.1))
#ax2.tick_params(axis='y', labelsize=16)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m'))
# Adjust layout
plt.tight_layout()

# Save the plot as a high-quality image (e.g., PNG or PDF)
plt.savefig("SoE_Plot_rulebased.png", dpi=300, bbox_inches="tight")
#plt.savefig("SoE_Plot_consq.png", dpi=300, bbox_inches="tight")
# Or, if you prefer to display the plot interactively, use plt.show()
# plt.show()


In [ ]:
# Data
power_tu1 = [power_tu1[t] for t in time]
power_tu2 = [power_tu2[t] for t in time]

# Create a new figure
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the data
ax.plot(profile.index[:len(power_tu1)], power_tu1, label="String_1_Power", color="orange", linewidth=2)
ax.plot(profile.index[:len(power_tu2)], power_tu2, label="String_2_Power", color="green", linewidth=1, linestyle='--')
# Set labels and title
ax.set_xlabel("Time",fontsize=24)
ax.set_ylabel("String Power (kW)",fontsize=24)
#ax.set_title("State of Charge Over Time")

# Add grid lines
ax.grid(True, linestyle='--')

# Add a legend
ax.legend(fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m'))

# Adjust layout
plt.tight_layout()

# Save the plot as a high-quality image (e.g., PNG or PDF)
plt.savefig("Power_split_equal.png", dpi=300, bbox_inches="tight")
# Or, if you prefer to display the plot interactively, use plt.show()
# plt.show()


In [ ]:
# Create a DataFrame from the lists
data = {'soc1': soc1, 'soc2': soc2, 'power_tu1': power_tu1, 'power_tu2': power_tu2, 'power_loss1': power_loss1, 'power_loss2': power_loss2 }
df = pd.DataFrame(data)

# Specify the path to your new CSV file
output_csv_file = 'output_ruleBased.csv'

# Save the DataFrame to a CSV file
df.to_csv(output_csv_file, index=False)

In [ ]:
total= sum(AI)
availability_index = total/ len(AI)
print("AI = ",availability_index)

In [ ]:
ave_soc1= sum(soc1)/len(soc1)
ave_soc2= sum(soc2)/len(soc2)
print("Average soc1:", ave_soc1)
print("Average soc2:", ave_soc2)
print("System Avergae SOC: ",(ave_soc1+ave_soc2)/2)

In [ ]:
output = pd.read_csv("output_ruleBased.csv")
profile = profile.drop(profile.index[-1])
total_input= sum(abs(profile.power))
print("total_input =", total_input*0.001)
total_system =sum(abs(output.power_tu1)) + sum(abs(output.power_tu2))
total_loss =sum(abs(output.power_loss1)) + sum(abs(output.power_loss2))
print("total_system =", total_system)

throughput= total_system/(total_input*0.001)
print("throughput =", throughput)
print("total_loss =", total_loss)
print("conversion =", 1-(total_loss/total_system))